# Few-Shot Learning with Amazon Bedrock

Welcome to our exploration of few-shot learning! In this notebook, we'll learn how to use examples to help our LLM understand exactly what we want. Think of it like teaching by showing examples - it's a powerful way to get more reliable and consistent results.

## Setup

Let's start with our usual imports:

In [ ]:
import boto3
import json

# Initialize the Bedrock client
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')

print("✅ Setup complete!")

## Helper Functions

We'll reuse the same helpers from the COT examples

In [ ]:
import re
from typing import Dict, Any, Optional

MODEL_ID: str = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
SYSTEM_PROMPT: str = "You are a helpful assistant"

# Helper function to call bedrock
def call_bedrock(prompt: str) -> str:

    # Create the message in Bedrock's required format
    user_message: Dict[str, Any] = { "role": "user","content": [{ "text": prompt}] }

    # Configure model parameters
    inference_config: Dict[str, Any] = {
        "temperature": .4,
        "maxTokens": 1000
    }

    # Send request to Claude Haiku 3.5 via Bedrock
    response: Dict[str, Any] = bedrock.converse(
        modelId=MODEL_ID,  # Using Sonnet 3.5 
        messages=[user_message],
        system=[{"text": SYSTEM_PROMPT}],
        inferenceConfig=inference_config
    )

    # Get the model's text response
    return response['output']['message']['content'][0]['text']

# Define a function to extract content from XML tags
def extract_tag_content(text: str, tag_name: str) -> Optional[str]:
    """
    Extract content between specified XML tags from text.
    """
    pattern = f'<{tag_name}>(.*?)</{tag_name}>'
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

## Basic Few-Shot Example

Let's start by comparing zero-shot (no examples) vs few-shot (with examples) prompting. We'll try to get the model to generate product descriptions in a specific format:

```
Product: <name>
Key Features: <numbered list of features>
Ideal For: <what it's ideal for>
Price Range: <price range>
```

In [ ]:
# Define the prompt text using XML tags for better Claude interaction
ZERO_SHOT_PROMPT = """
Generate a product description for a coffee maker. Describe the product name, key features, ideal for, and price range.
"""

# Send request to Claude
response: str = call_bedrock(ZERO_SHOT_PROMPT)

# Print the response
print("Zero-shot response:")
print(response)

**Results**

The results are good, but not quite what we were looking for format wise. It's also a bit verbose. Lets provide some examples of what we're looking for to improve the prompt

In [ ]:
# Few-shot prompt with examples
FEW_SHOT_PROMPT = """
Here are some examples of product descriptions in our preferred format:

<examples>
<response>
Product: Blender
Key Features:
1. 1000W motor
2. 6 speed settings
3. 64 oz capacity
Ideal For: Smoothies, soups, sauces
Price Range: $$$
</response>

<response>
Product: Toaster
Key Features:
1. 4 slice capacity
2. Digital controls
3. 7 browning levels
Ideal For: Breakfast, quick meals
Price Range: $$
</response>
</examples>

Now, generate a product description for a coffee maker in the same format and place your response between the <response> tags."
"""

model_response: str = call_bedrock(FEW_SHOT_PROMPT)

response: str = extract_tag_content(model_response, "response")

print("Few-shot response:\n\n")
print(response)

Notice how the few-shot example produced a much more structured and consistent format! This is because we showed the model exactly what we wanted through examples. The models are also pretty chatty, so it's still useful to get them to respond in response tags that we can regex out. 

## More Complex Few-Shot Learning

Now let's try something more complex - teaching the model to classify customer feedback into specific categories and in a specific format.

In [ ]:
CATEGORIZATION_FEW_SHOT_PROMPT_TEMPLATE = """
Here are some examples of categorized customer feedback in the preferred response format:

<examples>
Feedback: The website kept crashing while I tried to check out
Category: Technical Issue
Priority: High

Feedback: I love the new design, it's so much easier to find things
Category: UI/UX
Priority: Low

Feedback: My order arrived damaged and customer service hasn't responded in 3 days
Category: Customer Service
Priority: High
</examples>

Now, classify the following customer feedback into a category and priority:
<feedback>
{feedback}
</feedback>

When responding, place your response between the <response> tags in that format.
"""

# Create inputs for our template
inputs: Dict[str, str] = { "feedback": "I've been waiting for a refund for two weeks now and can't get anyone to help me." }

# Format the prompt with our inputs
prompt: str = CATEGORIZATION_FEW_SHOT_PROMPT_TEMPLATE.format(**inputs)

model_response: str = call_bedrock(prompt)

response: str = extract_tag_content(model_response, "response")

print("Few-shot response:\n\n")
print(response)

## Tips for Effective Few-Shot Learning

1. Use more than 1 example for best results. You start seeing diminishing returns (and increased costs after 10)
2. Make your examples diverse but representative
3. Keep the format consistent across examples
5. Use realistic examples that match your use case

## Advanced
The examples don't have to be static. You can dynamically pass in the most relevant examples into the prompt through prompt variables.


## Exercise

Now it's your turn! Try writing a few-shot prompt to teach the model to generate recipe instructions in this format:

```
Recipe: [name]
Prep Time: [time]
Difficulty: [easy/medium/hard]
Steps:
1. [step]
2. [step]
...
```

Create 2-3 example recipes, then ask for a new recipe. How well does the model follow your format?